In [4]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary
try:
    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1
y 0
z 1
Obj: 3


In [15]:
import gurobipy as gp
from gurobipy import GRB

# Sample Data
items = ['item1', 'item2', 'item3', 'item4', 'item5']
utilities = [0.9, 0.8, 0.7, 0.6, 0.5]
categories = ['A', 'A', 'B', 'B', 'C']
user_preferences = {'A': 0.4, 'B': 0.4, 'C': 0.2}
N = 3  # Number of items to recommend

# Category mapping
category_items = {
    'A': [0, 1],
    'B': [2, 3],
    'C': [4]
}

# Initialize Model
model = gp.Model('calibrated_reranking')

# Decision Variables
x = model.addVars(len(items), vtype=GRB.BINARY, name='x')

# Objective Function
model.setObjective(gp.quicksum(utilities[i] * x[i] for i in range(len(items))), GRB.MAXIMIZE)

# Calibration Constraints
for cat, proportion in user_preferences.items():
    num_items_cat = proportion * N
    for i in category_items[cat]:
        print(i)
    model.addConstr(
    gp.quicksum(x[i] for i in category_items[cat]) >= int(num_items_cat),
    name=f'calibration_lower_{cat}'
    )
    model.addConstr(
        gp.quicksum(x[i] for i in category_items[cat]) <= int(num_items_cat) + 1,
        name=f'calibration_upper_{cat}'
    )

# Total Recommendation Constraint
model.addConstr(gp.quicksum(x[i] for i in range(len(items))) == N, name='total_recommendations')

# Optimize
model.optimize()

# Display Results
if model.status == GRB.OPTIMAL:
    selected_items = [items[i] for i in range(len(items)) if x[i].x > 0.5]
    print("Selected items:", selected_items)
    print("Total Utility:", model.objVal)
else:
    print("No optimal solution found.")


0
1
2
3
4
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 15 nonzeros
Model fingerprint: 0xc0862072
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 2.4000000
Presolve removed 7 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2.4 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.400000000000e+00, best bound 2.400000000000e+00, gap 0.0000%
Selected items: ['item1', 'item2', 'item3']
Total Utility: 2.4000000000000004


In [12]:
import gurobipy as gp
from gurobipy import GRB

# Sample Data
items = ['item1', 'item2', 'item3', 'item4', 'item5']
utilities = [0.9, 0.8, 0.7, 0.6, 0.5]
categories = {'A': [1, 1, 0, 0, 1], 'B': [0, 1, 1, 1, 0], 'C': [0, 0, 1, 1, 1]}  # Category membership matrix
user_preferences = {'A': 0.4, 'B': 0.4, 'C': 0.2}
N = 3  # Number of items to recommend

# Initialize Model
model = gp.Model('calibrated_reranking')

# Decision Variables
x = model.addVars(len(items), vtype=GRB.BINARY, name='x')

# Objective Function
model.setObjective(gp.quicksum(utilities[i] * x[i] for i in range(len(items))), GRB.MAXIMIZE)

# Calibration Constraints
for cat, membership in categories.items():
    
    proportion = user_preferences[cat]
    num_items_cat = proportion * N
    print(proportion)
    print(num_items_cat)
    model.addConstr(
        gp.quicksum(membership[i] * x[i] for i in range(len(items))) == int(num_items_cat),
        name=f'calibration_{cat}'
    )

# Total Recommendation Constraint
model.addConstr(gp.quicksum(x[i] for i in range(len(items))) == N, name='total_recommendations')

# Optimize
model.optimize()

# Display Results
if model.status == GRB.OPTIMAL:
    selected_items = [items[i] for i in range(len(items)) if x[i].x > 0.5]
    print("Selected items:", selected_items)
    print("Total Utility:", model.objVal)
else:
    print("No optimal solution found.")


0.4
1.2000000000000002
0.4
1.2000000000000002
0.2
0.6000000000000001
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 5 columns and 14 nonzeros
Model fingerprint: 0x32f8b301
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.


In [13]:
import gurobipy as gp
from gurobipy import GRB

# Sample Data
items = ['item1', 'item2', 'item3', 'item4', 'item5']
utilities = [0.9, 0.8, 0.7, 0.6, 0.5]
category_contributions = {
    'A': [0.5, 0.0, 0.5, 0.33, 0.0],
    'B': [0.5, 1.0, 0.0, 0.33, 0.0],
    'C': [0.0, 0.0, 0.5, 0.33, 1.0]
}
user_preferences = {'A': 0.4, 'B': 0.4, 'C': 0.2}
N = 3  # Number of items to recommend

# Initialize Model
model = gp.Model('calibrated_reranking')

# Decision Variables
x = model.addVars(len(items), vtype=GRB.BINARY, name='x')

# Objective Function
model.setObjective(gp.quicksum(utilities[i] * x[i] for i in range(len(items))), GRB.MAXIMIZE)

# Calibration Constraints
for cat, contributions in category_contributions.items():
    proportion = user_preferences[cat]
    num_items_cat = proportion * N
    model.addConstr(
        gp.quicksum(contributions[i] * x[i] for i in range(len(items))) == num_items_cat,
        name=f'calibration_{cat}'
    )

# Total Recommendation Constraint
model.addConstr(gp.quicksum(x[i] for i in range(len(items))) == N, name='total_recommendations')

# Optimize
model.optimize()

# Display Results
if model.status == GRB.OPTIMAL:
    selected_items = [items[i] for i in range(len(items)) if x[i].x > 0.5]
    print("Selected items:", selected_items)
    print("Total Utility:", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 5 columns and 14 nonzeros
Model fingerprint: 0xb45b2fa6
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [5e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 3e+00]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.


In [20]:
x=[1,0,1]
for i in range(len(items)):
    gp.quicksum(contributions[i] * x[i]) 

TypeError: 'float' object is not iterable

In [23]:
for cat, contributions in category_contributions.items():
    print(contributions)

[0.5, 0.0, 0.5, 0.33, 0.0]
[0.5, 1.0, 0.0, 0.33, 0.0]
[0.0, 0.0, 0.5, 0.33, 1.0]


In [76]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Sample data
N = 4  # Number of items
M = 3   # Number of categories (A, B, C)
scores =[2,3,5,5]  # Random item scores
category_proportions = np.array([[0.32923741, 0.35103552, 0.31972708],
       [0.2960835 , 0.49735459, 0.20656191],
       [0.15276603, 0.45660195, 0.39063202],[0.25276603, 0.35660195, 0.39063202],])

# Constant category distribution
constant_category_dist = np.array([0.1, 0.4, 0.5])

# Create a Gurobi model
model = gp.Model()

# Define binary variables xi (1 if item i is selected, 0 otherwise)
xi = model.addVars(N, vtype=GRB.BINARY, name="xi")

# Define category sums
category_sums = model.addVars(M, name="category_sums")  # Sum of each category

# Define constraints for category sums
for c in range(M):
    model.addConstr(category_sums[c] == gp.quicksum(xi[i] * category_proportions[i, c] for i in range(N)))
epsilon = 1e-5
total_sum = model.addVar(name="total_sum", lb=epsilon)  # Ensure total_sum is always positive
model.addConstr(total_sum == gp.quicksum(category_sums[c] for c in range(M)))
P = model.addVars(M, name="P", lb=0)
for c in range(M):
    model.addConstr(P[c] == category_sums[c] )

# Add auxiliary variables for KLD terms
log_terms = model.addVars(M, name="log_terms")
for c in range(M):
    # Avoid division by zero and log of zero
    model.addGenConstrLog(P[c], log_terms[c])  # log(P[c])

# Calculate KLD: KLD(P || Q) = sum(P(c) * log(P(c) / Q(c)))
kld_terms = gp.quicksum(P[c] * (log_terms[c] - np.log(constant_category_dist[c])) for c in range(M))

# Objective function: Maximize score and minimize KLD
lambda_kld = 0.2  # Weight for KLD term
# model.addConstr(gp.quicksum(xi[i] for i in range(len(category_proportions))) == 2, name='total_recommendations')

model.setObjective(gp.quicksum(scores[i] * xi[i] for i in range(N)) - lambda_kld * kld_terms, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Extract the results
if model.status == GRB.OPTIMAL:
    selected_items = [i for i in range(N) if xi[i].x > 0.5]
    print("Selected items:", selected_items)
    print("Objective value:", model.objVal)
    print("Recommended category distribution:")
    for c in range(M):
        print(f"Category {c}: {P[c].x:.4f}")
    for i in range(M):
        print(f"items {i}: {log_terms[i].x:.4f}")
else:
    print("No optimal solution found.")
    if model.status == GRB.INFEASIBLE:
        print("Model is infeasible. Writing IIS (Irreducible Inconsistent Subsystem) to file.")
        model.computeIIS()
        model.write("infeasible_model.ilp")



Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 14 columns and 25 nonzeros
Model fingerprint: 0x43259ac0
Model has 3 quadratic objective terms
Model has 3 function constraints treated as nonlinear
  3 LOG
Variable types: 10 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e-01, 5e+00]
  QObjective range [4e-01, 4e-01]
  Bounds range     [1e-05, 1e+00]
  RHS range        [0e+00, 0e+00]
         Setting lower bound to 1e-10.
Presolve removed 7 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 13.7944 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.379437386856e+01, best bound 1.3794

In [86]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Sample data
N = 8  # Number of items
M = 3  # Number of categories (A, B, C)
scores = [2, 3, 5, 5, 4, 6, 7, 8]  # Updated scores for 8 items
category_proportions = np.array([
    [0.329, 0.351, 0.319],
    [0.296, 0.497, 0.206],
    [0.153, 0.457, 0.391],
    [0.253, 0.357, 0.390],
    [0.200, 0.300, 0.500],
    [0.150, 0.500, 0.350],
    [0.400, 0.300, 0.300],
    [0.250, 0.400, 0.350]
])

# Constant category distribution
constant_category_dist = np.array([0.1, 0.4, 0.5])

# Create a Gurobi model
model = gp.Model("Item_Selection_More_Items")

# Binary variables for item selection
xi = model.addVars(N, vtype=GRB.BINARY, name="xi")

# Category sums
category_sums = model.addVars(M, name="category_sums")

# Total sum of selected items
epsilon = 1e-5  # Small constant to avoid division by zero
total_sum = model.addVar(name="total_sum", lb=epsilon)

# Ensure category sums match selected items
for c in range(M):
    model.addConstr(category_sums[c] == gp.quicksum(xi[i] * category_proportions[i, c] for i in range(N)),
                    name=f'category_sum_{c}')

# Total category sum constraint
model.addConstr(total_sum == gp.quicksum(category_sums[c] for c in range(M)), name="total_sum_constraint")

# Define proportions P
P = model.addVars(M, name="P", lb=epsilon)
for c in range(M):
    model.addConstr(P[c] == category_sums[c] / total_sum, name=f'proportion_{c}')

# Log terms for KLD calculation
log_terms = model.addVars(M, name="log_terms")
for c in range(M):
    model.addGenConstrLog(P[c], log_terms[c])  # log(P[c])

# KLD Objective
kld_terms = gp.quicksum(P[c] * (log_terms[c] - np.log(constant_category_dist[c])) for c in range(M))

# Add constraint for total selected items (e.g., select exactly 3 items)
# model.addConstr(gp.quicksum(xi[i] for i in range(N)) == 3, name='total_recommendations')

# Objective: Maximize score and minimize KLD
lambda_kld = 0.2  # Weight for KLD term
model.setObjective(gp.quicksum(scores[i] * xi[i] for i in range(N)) - lambda_kld * kld_terms, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Check the status of the model
if model.status == GRB.OPTIMAL:
    selected_items = [i for i in range(N) if xi[i].x > 0.5]
    print("Selected items:", selected_items)
    print("Objective value:", model.objVal)
    print("Recommended category distribution:")
    for c in range(M):
        print(f"Category {c}: {P[c].x:.4f}")
    for i in range(N):
        print(f"Item {i}: {xi[i].x:.4f}")
elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Writing IIS (Irreducible Inconsistent Subsystem) to file.")
    model.computeIIS()
    model.write("infeasible_model.ilp")
elif model.status == GRB.UNBOUNDED:
    print("Model is unbounded. Please review the objective and constraints.")
else:
    print(f"Optimization ended with status {model.status}.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 18 columns and 31 nonzeros
Model fingerprint: 0x8109dad3
Model has 3 quadratic objective terms
Model has 3 function constraints treated as nonlinear
  3 LOG
Model has 3 general nonlinear constraints (3 nonlinear terms)
Variable types: 10 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e-01, 8e+00]
  QObjective range [4e-01, 4e-01]
  Bounds range     [1e-05, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve model has 6 nlconstr
Added 3 variables to disaggregate expressions.
Presolve time: 0.00s
Presolved: 65 rows, 26 columns, 170 nonzeros
Presolved model has 9 bilinear constraint(s)
Presolved model has 6 nonlinear constraint(s)

Solving non-convex MINLP

Variable types: 18 continuous, 8 integer (8 binary)

Explored 

In [87]:
model.computeIIS()
model.write("model.ilp")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

IIS computation: initial model status unknown, solving to determine model status
Presolve model has 6 nlconstr
Added 3 variables to disaggregate expressions.
Presolve time: 0.00s
Presolved: 58 rows, 22 columns, 151 nonzeros
Presolved model has 6 bilinear constraint(s)
Presolved model has 6 nonlinear constraint(s)

Solving non-convex MINLP

Variable types: 14 continuous, 8 integer (8 binary)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -

Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Gue

In [92]:
for c in model.getConstrs():
    if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

for v in model.getVars():
    if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
    if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')

AttributeError: Unable to retrieve attribute 'IISConstr'

In [97]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Sample data
N = 8  # Number of items
M = 3  # Number of categories (A, B, C)
scores = [2, 3, 5, 5, 4, 6, 7, 8]  # Updated scores for 8 items
category_proportions = np.array([
    [0.329, 0.351, 0.319],
    [0.296, 0.497, 0.206],
    [0.153, 0.457, 0.391],
    [0.253, 0.357, 0.390],
    [0.200, 0.300, 0.500],
    [0.150, 0.500, 0.350],
    [0.400, 0.300, 0.300],
    [0.250, 0.400, 0.350]
])

# Constant category distribution
constant_category_dist = np.array([0.1, 0.4, 0.5])

# Create a Gurobi model
model = gp.Model("Item_Selection_More_Items")

# Binary variables for item selection
xi = model.addVars(N, vtype=GRB.BINARY, name="xi")

# Category sums
category_sums = model.addVars(M, lb=0, name="category_sums")

# Total sum of selected items
epsilon = 1e-5  # Small constant to avoid division by zero
total_sum = model.addVar(name="total_sum", lb=epsilon)

# Ensure category sums match selected items
for c in range(M):
    model.addConstr(category_sums[c] == gp.quicksum(xi[i] * category_proportions[i, c] for i in range(N)),
                    name=f'category_sum_{c}')

# Total category sum constraint
model.addConstr(total_sum == gp.quicksum(category_sums[c] for c in range(M)), name="total_sum_constraint")

# Define proportions P
P = model.addVars(M, name="P", lb=epsilon)
for c in range(M):
    model.addConstr(P[c] == category_sums[c] / total_sum, name=f'proportion_{c}')

log_terms = model.addVars(M, name="log_terms")
for c in range(M):
    model.addGenConstrLog(P[c], log_terms[c], name=f"{c}_log")  


kld_terms = gp.quicksum(constant_category_dist[c] * (np.log(constant_category_dist[c])-log_terms[c] ) for c in range(M))

model.addConstr(gp.quicksum(xi[i] for i in range(N)) == 3, name='total_recommendations')

lambda_kld = 0
model.setObjective(gp.quicksum(scores[i] * xi[i] for i in range(N)) - lambda_kld * kld_terms, GRB.MAXIMIZE)

# Optimize the model
model.reset()
# model., 0)
model.optimize()

# Check the status of the model
if model.status == GRB.OPTIMAL:
    selected_items = [i for i in range(N) if xi[i].x > 0.5]
    print("Selected items:", selected_items)
    print("Objective value:", model.objVal)
    print("Recommended category distribution:")
    for c in range(M):
        print(f"Category {c}: {P[c].x:.4f}")
    for i in range(N):
        print(f"Item {i}: {xi[i].x:.4f}")
elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Writing IIS (Irreducible Inconsistent Subsystem) to file.")
    model.computeIIS()
    model.write("infeasible_model.ilp")
elif model.status == GRB.UNBOUNDED:
    print("Model is unbounded. Please review the objective and constraints.")
else:
    print(f"Optimization ended with status {model.status}.")


Discarded solution information
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 18 columns and 39 nonzeros
Model fingerprint: 0xeea36ccf
Model has 3 function constraints treated as nonlinear
  3 LOG
Model has 3 general nonlinear constraints (3 nonlinear terms)
Variable types: 10 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e-05, 1e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -
Model is infeasible. Writing IIS (Irreducible Inconsistent Subsystem) to file.
Gurobi Optimizer version 1

In [98]:
from gurobipy import Model, GRB, quicksum

# Data (placeholder)
N = 5  # Number of movies to recommend
movies = range(10)  # 10 candidate movies
s = [1.2, 0.8, 0.5, 1.7, 1.0, 0.9, 1.3, 0.6, 1.5, 0.7]  # Scores for each movie
w_r = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05]  # Weights
lambda_ = 0.5

# Model
model = Model("calibrated_recommendation")

# Variables: Binary selection for each movie
x = model.addVars(movies, vtype=GRB.BINARY, name="x")

# Constraint: Select exactly N movies
model.addConstr(quicksum(x[i] for i in movies) == N, name="size_constraint")

# Objective: (1-λ) * Score - λ * Calibration
# Approximate calibration term with linearization (simplified here)
score_term = quicksum(s[i] * x[i] for i in movies)
log_sum_term = model.addVar(lb=-GRB.INFINITY, name="log_sum_term")

# Approximate log(sum(w_r * x))
model.addGenConstrLog(log_sum_term, quicksum(w_r[i] * x[i] for i in movies), "log_constraint")

# Final objective
model.setObjective((1 - lambda_) * score_term - lambda_ * log_sum_term, GRB.MAXIMIZE)

# Optimize
model.optimize()

# Display Results
for i in movies:
    if x[i].X > 0.5:
        print(f"Selected Movie: {i}")


TypeError: 0.9 <gurobi.Var *Awaiting Model Update*> + 0.8 <gurobi.Var *Awaiting Model Update*> + 0.7 <gurobi.Var *Awaiting Model Update*> + 0.6 <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*> + 0.4 <gurobi.Var *Awaiting Model Update*> + 0.3 <gurobi.Var *Awaiting Model Update*> + 0.2 <gurobi.Var *Awaiting Model Update*> + 0.1 <gurobi.Var *Awaiting Model Update*> + 0.05 <gurobi.Var *Awaiting Model Update*> is not a variable